In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import yfinance as yf
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import pandas_ta as ta
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [3]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [4]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]

In [5]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M30
start_datetime = datetime(2019,12,15)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# hour4 df
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# daily df
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df
hourly_df
ohlc_df

,time,open,high,low,close
0,2019-12-16 00:00:00,1.11247,1.11279,1.11243,1.11255
1,2019-12-16 00:30:00,1.11255,1.11303,1.11246,1.11286
2,2019-12-16 01:00:00,1.11286,1.11296,1.11226,1.11253
3,2019-12-16 01:30:00,1.11253,1.11281,1.11236,1.11242
4,2019-12-16 02:00:00,1.11243,1.11267,1.11230,1.11237
...,...,...,...,...,...
70082,2025-08-01 21:30:00,1.15396,1.15479,1.15254,1.15258
70083,2025-08-01 22:00:00,1.15258,1.15457,1.15236,1.15441
70084,2025-08-01 22:30:00,1.15441,1.15823,1.15440,1.15689
70085,2025-08-01 23:00:00,1.15692,1.15897,1.15688,1.15893


In [6]:
#ohlc4
daily_df['ohlc'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
hourly_df['ohlc'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4
#sma
daily_df['sma_1'] = ta.sma(close = daily_df['ohlc'], length= 1 )
daily_df['sma_14'] = ta.sma(close = daily_df['ohlc'], length= 14)
hourly_df['sma_11'] = ta.sma(close = hourly_df['ohlc'], length= 1)
hourly_df['sma_144'] = ta.sma(close = hourly_df['ohlc'], length= 14)
daily_df['high_20'] = ta.sma(close = daily_df['high'], length= 20 )
daily_df['low_20'] = ta.sma(close = daily_df['low'], length= 20)
daily_df

,time,open,high,low,close,ohlc,sma_1,sma_14,high_20,low_20
0,2019-12-16,1.11247,1.11580,1.11226,1.11440,1.113733,1.113733,NaN,NaN,NaN
1,2019-12-17,1.11430,1.11746,1.11293,1.11508,1.114942,1.114942,NaN,NaN,NaN
2,2019-12-18,1.11509,1.11542,1.11103,1.11134,1.113220,1.113220,NaN,NaN,NaN
3,2019-12-19,1.11131,1.11443,1.11072,1.11202,1.112120,1.112120,NaN,NaN,NaN
4,2019-12-20,1.11218,1.11248,1.10663,1.10764,1.109733,1.109733,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1456,2025-07-28,1.17552,1.17704,1.15838,1.15885,1.167448,1.167448,1.168465,1.174926,1.166657
1457,2025-07-29,1.15875,1.15994,1.15189,1.15452,1.156275,1.156275,1.167368,1.173774,1.165444
1458,2025-07-30,1.15461,1.15728,1.14002,1.14031,1.148055,1.148055,1.165747,1.172588,1.163710
1459,2025-07-31,1.14033,1.14610,1.13991,1.14135,1.141923,1.141923,1.163806,1.170842,1.162119


In [7]:
def entring_stoploss_profit(row, hourly_df) :
    if row["sma_1"] > row['sma_14'] and row["sma_11"] > row['sma_144'] :
        return "buy"
    elif row['sma_1'] < row['sma_14'] and row['sma_11'] < row['sma_144'] :
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df1 = pd.merge_asof(ohlc_df, hourly_df, on='time', direction='backward')
merged_df = pd.merge_asof(merged_df1, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, hourly_df = hourly_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

,signal,time
628,buy,2020-01-06 08:00:00
660,buy,2020-01-07 00:00:00
708,sell,2020-01-08 00:00:00
756,sell,2020-01-09 00:00:00
804,sell,2020-01-10 00:00:00
...,...,...
69863,sell,2025-07-28 08:00:00
69895,sell,2025-07-29 00:00:00
69943,sell,2025-07-30 00:00:00
69991,sell,2025-07-31 00:00:00


In [8]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "hourly_low",
    "high_y": "hourly_high",
    "open_y": "hourly_open",
    "close_y": "hourly_close",
    "low": "daily_low",
    "high": "daily_high",
    "open": "daily_open",
    "close": "daily_close"
})


In [9]:
print(merged_df.columns.tolist())
merged_df

['time', 'open', 'high', 'low', 'close', 'hourly_open', 'hourly_high', 'hourly_low', 'hourly_close', 'ohlc_x', 'sma_11', 'sma_144', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'ohlc_y', 'sma_1', 'sma_14', 'high_20', 'low_20', 'signal', 'is_first_signal']


,time,open,high,low,close,hourly_open,hourly_high,hourly_low,hourly_close,ohlc_x,...,daily_high,daily_low,daily_close,ohlc_y,sma_1,sma_14,high_20,low_20,signal,is_first_signal
0,2019-12-16 00:00:00,1.11247,1.11279,1.11243,1.11255,1.11247,1.11303,1.11226,1.11281,1.112643,...,1.11580,1.11226,1.11440,1.113733,1.113733,NaN,NaN,NaN,None,NaN
1,2019-12-16 00:30:00,1.11255,1.11303,1.11246,1.11286,1.11247,1.11303,1.11226,1.11281,1.112643,...,1.11580,1.11226,1.11440,1.113733,1.113733,NaN,NaN,NaN,None,NaN
2,2019-12-16 01:00:00,1.11286,1.11296,1.11226,1.11253,1.11247,1.11303,1.11226,1.11281,1.112643,...,1.11580,1.11226,1.11440,1.113733,1.113733,NaN,NaN,NaN,None,NaN
3,2019-12-16 01:30:00,1.11253,1.11281,1.11236,1.11242,1.11247,1.11303,1.11226,1.11281,1.112643,...,1.11580,1.11226,1.11440,1.113733,1.113733,NaN,NaN,NaN,None,NaN
4,2019-12-16 02:00:00,1.11243,1.11267,1.11230,1.11237,1.11247,1.11303,1.11226,1.11281,1.112643,...,1.11580,1.11226,1.11440,1.113733,1.113733,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70082,2025-08-01 21:30:00,1.15396,1.15479,1.15254,1.15258,1.15393,1.15971,1.15165,1.15847,1.155940,...,1.15971,1.13917,1.15847,1.149633,1.149633,1.162574,1.16989,1.160327,None,NaN
70083,2025-08-01 22:00:00,1.15258,1.15457,1.15236,1.15441,1.15393,1.15971,1.15165,1.15847,1.155940,...,1.15971,1.13917,1.15847,1.149633,1.149633,1.162574,1.16989,1.160327,None,NaN
70084,2025-08-01 22:30:00,1.15441,1.15823,1.15440,1.15689,1.15393,1.15971,1.15165,1.15847,1.155940,...,1.15971,1.13917,1.15847,1.149633,1.149633,1.162574,1.16989,1.160327,None,NaN
70085,2025-08-01 23:00:00,1.15692,1.15897,1.15688,1.15893,1.15393,1.15971,1.15165,1.15847,1.155940,...,1.15971,1.13917,1.15847,1.149633,1.149633,1.162574,1.16989,1.160327,None,NaN


In [10]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = (data['high'] - data["low_20"]) # example SL
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = (data['high_20'] - data["low_20"]) # example SL
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["sma_11"] <= data["sma_144"]:
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data["sma_11"] >= data["sma_144"]:
            orders.close_trade(trade)

In [11]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,100000,2020-01-06 08:00:00,1.11624,2020-01-07 12:00:00,1.11846,0,0,{},222.0,-7.0,215.0,215.0,10215.0
1,closed,EURUSD,sell,100000,2020-01-08 00:00:00,1.11519,2020-01-10 20:00:00,1.11224,0,0,{},295.0,-7.0,288.0,503.0,10503.0
2,closed,EURUSD,sell,100000,2020-01-14 12:00:00,1.11314,2020-01-14 20:00:00,1.1131,0,0,{},4.0,-7.0,-3.0,500.0,10500.0
3,closed,EURUSD,sell,100000,2020-01-16 20:00:00,1.11348,2020-01-21 12:00:00,1.10907,0,0,{},441.0,-7.0,434.0,934.0,10934.0
4,closed,EURUSD,sell,100000,2020-01-22 00:00:00,1.10818,2020-01-23 12:00:00,1.10844,0,0,{},-26.0,-7.0,-33.0,901.0,10901.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,closed,EURUSD,buy,100000,2025-07-07 00:00:00,1.17771,2025-07-07 04:00:00,1.17797,0,0,{},26.0,-7.0,19.0,182263.0,192263.0
512,closed,EURUSD,sell,100000,2025-07-10 12:00:00,1.17279,2025-07-15 08:00:00,1.16759,0,0,{},520.0,-7.0,513.0,182776.0,192776.0
513,closed,EURUSD,sell,100000,2025-07-16 00:00:00,1.16005,2025-07-18 04:00:00,1.16246,0,0,{},-241.0,-7.0,-248.0,182528.0,192528.0
514,closed,EURUSD,buy,100000,2025-07-22 00:00:00,1.16894,2025-07-25 00:00:00,1.17442,0,0,{},548.0,-7.0,541.0,183069.0,193069.0


In [12]:
mm = bt.plot_balance()
display(mm)

In [13]:
evaluate_backtest(bt.trades)

biggest_profit: 4713.0
daily_drawdown: -1622.0
max_drawown: -1629.0
avg_win: 613.83
avg_loss: -158.46
count_profit_trades: 352
count_loss_trades: 164
Win Rate1: 68.22%
rrr: 3.87


,order_type,profit
0,buy,104760.0
1,sell,85322.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,current_max,drawdown
0,closed,EURUSD,buy,100000,2020-01-06 08:00:00,1.11624,2020-01-07 12:00:00,1.11846,0,0,{},222.0,-7.0,215.0,215.0,10215.0,215.0,0.0
1,closed,EURUSD,sell,100000,2020-01-08 00:00:00,1.11519,2020-01-10 20:00:00,1.11224,0,0,{},295.0,-7.0,288.0,503.0,10503.0,503.0,0.0
2,closed,EURUSD,sell,100000,2020-01-14 12:00:00,1.11314,2020-01-14 20:00:00,1.1131,0,0,{},4.0,-7.0,-3.0,500.0,10500.0,503.0,-3.0
3,closed,EURUSD,sell,100000,2020-01-16 20:00:00,1.11348,2020-01-21 12:00:00,1.10907,0,0,{},441.0,-7.0,434.0,934.0,10934.0,934.0,0.0
5,closed,EURUSD,sell,100000,2020-01-24 00:00:00,1.10531,2020-01-30 04:00:00,1.10112,0,0,{},419.0,-7.0,412.0,1313.0,11313.0,1313.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,closed,EURUSD,buy,100000,2025-06-23 16:00:00,1.14676,2025-07-02 12:00:00,1.17757,0,0,{},3081.0,-7.0,3074.0,182255.0,192255.0,182255.0,0.0
511,closed,EURUSD,buy,100000,2025-07-07 00:00:00,1.17771,2025-07-07 04:00:00,1.17797,0,0,{},26.0,-7.0,19.0,182263.0,192263.0,182263.0,0.0
512,closed,EURUSD,sell,100000,2025-07-10 12:00:00,1.17279,2025-07-15 08:00:00,1.16759,0,0,{},520.0,-7.0,513.0,182776.0,192776.0,182776.0,0.0
514,closed,EURUSD,buy,100000,2025-07-22 00:00:00,1.16894,2025-07-25 00:00:00,1.17442,0,0,{},548.0,-7.0,541.0,183069.0,193069.0,183069.0,0.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,current_max,drawdown
4,closed,EURUSD,sell,100000,2020-01-22 00:00:00,1.10818,2020-01-23 12:00:00,1.10844,0,0,{},-26.0,-7.0,-33.0,901.0,10901.0,934.0,-33.0
6,closed,EURUSD,buy,100000,2020-02-03 00:00:00,1.10932,2020-02-04 08:00:00,1.10585,0,0,{},-347.0,-7.0,-354.0,959.0,10959.0,1313.0,-354.0
8,closed,EURUSD,sell,100000,2020-02-21 00:00:00,1.07846,2020-02-21 08:00:00,1.07931,0,0,{},-85.0,-7.0,-92.0,3255.0,13255.0,3347.0,-92.0
11,closed,EURUSD,sell,100000,2020-03-16 00:00:00,1.10635,2020-03-16 12:00:00,1.12257,0,0,{},-1622.0,-7.0,-1629.0,6624.0,16624.0,8253.0,-1629.0
13,closed,EURUSD,buy,100000,2020-03-27 00:00:00,1.10303,2020-03-30 16:00:00,1.10152,0,0,{},-151.0,-7.0,-158.0,11172.0,21172.0,11330.0,-158.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,closed,EURUSD,sell,100000,2025-05-15 20:00:00,1.1181,2025-05-16 04:00:00,1.12049,0,0,{},-239.0,-7.0,-246.0,176669.0,186669.0,176915.0,-246.0
499,closed,EURUSD,sell,100000,2025-05-19 00:00:00,1.11755,2025-05-19 08:00:00,1.11838,0,0,{},-83.0,-7.0,-90.0,176579.0,186579.0,176915.0,-336.0
508,closed,EURUSD,buy,100000,2025-06-20 00:00:00,1.14918,2025-06-23 00:00:00,1.14562,0,0,{},-356.0,-7.0,-363.0,179181.0,189181.0,179544.0,-363.0
510,closed,EURUSD,buy,100000,2025-07-03 00:00:00,1.17983,2025-07-03 12:00:00,1.17979,0,0,{},-4.0,-7.0,-11.0,182244.0,192244.0,182255.0,-11.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,commission,profit_net,profit_cumulative,balance,current_max,drawdown,dayofweek,hourofday,month,year
0,closed,EURUSD,buy,100000,2020-01-06 08:00:00,1.11624,2020-01-07 12:00:00,1.11846,0,0,...,-7.0,215.0,215.0,10215.0,215.0,0.0,0,8,2020-01,2020
1,closed,EURUSD,sell,100000,2020-01-08 00:00:00,1.11519,2020-01-10 20:00:00,1.11224,0,0,...,-7.0,288.0,503.0,10503.0,503.0,0.0,2,0,2020-01,2020
2,closed,EURUSD,sell,100000,2020-01-14 12:00:00,1.11314,2020-01-14 20:00:00,1.1131,0,0,...,-7.0,-3.0,500.0,10500.0,503.0,-3.0,1,12,2020-01,2020
3,closed,EURUSD,sell,100000,2020-01-16 20:00:00,1.11348,2020-01-21 12:00:00,1.10907,0,0,...,-7.0,434.0,934.0,10934.0,934.0,0.0,3,20,2020-01,2020
4,closed,EURUSD,sell,100000,2020-01-22 00:00:00,1.10818,2020-01-23 12:00:00,1.10844,0,0,...,-7.0,-33.0,901.0,10901.0,934.0,-33.0,2,0,2020-01,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,closed,EURUSD,buy,100000,2025-07-07 00:00:00,1.17771,2025-07-07 04:00:00,1.17797,0,0,...,-7.0,19.0,182263.0,192263.0,182263.0,0.0,0,0,2025-07,2025
512,closed,EURUSD,sell,100000,2025-07-10 12:00:00,1.17279,2025-07-15 08:00:00,1.16759,0,0,...,-7.0,513.0,182776.0,192776.0,182776.0,0.0,3,12,2025-07,2025
513,closed,EURUSD,sell,100000,2025-07-16 00:00:00,1.16005,2025-07-18 04:00:00,1.16246,0,0,...,-7.0,-248.0,182528.0,192528.0,182776.0,-248.0,2,0,2025-07,2025
514,closed,EURUSD,buy,100000,2025-07-22 00:00:00,1.16894,2025-07-25 00:00:00,1.17442,0,0,...,-7.0,541.0,183069.0,193069.0,183069.0,0.0,1,0,2025-07,2025


max_drawdown -1629.0
